## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hohr-Grenzhausen,DE,50.4333,7.6667,68.04,92,13,3.65,few clouds
1,1,Narsaq,GL,60.9167,-46.0500,59.22,53,42,4.36,scattered clouds
2,2,Mahebourg,MU,-20.4081,57.7000,70.09,64,20,11.50,few clouds
3,3,Hithadhoo,MV,-0.6000,73.0833,80.60,75,87,4.00,light rain
4,4,Hilo,US,19.7297,-155.0900,82.80,85,90,4.61,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Hithadhoo,MV,-0.6000,73.0833,80.60,75,87,4.00,light rain
4,4,Hilo,US,19.7297,-155.0900,82.80,85,90,4.61,light rain
6,6,Trairi,BR,-3.2778,-39.2689,80.28,68,17,12.71,few clouds
7,7,Broome,US,42.2506,-75.8330,85.69,64,40,3.00,scattered clouds
8,8,Butaritari,KI,3.0707,172.7902,82.08,74,92,11.12,moderate rain
12,12,Faanui,PF,-16.4833,-151.7500,78.01,66,0,12.86,clear sky
13,13,Vaini,TO,-21.2000,-175.2000,75.47,83,71,4.83,broken clouds
14,14,Albany,US,42.6001,-73.9662,85.03,75,60,1.99,broken clouds
15,15,Kapaa,US,22.0752,-159.3190,82.72,79,75,1.99,light rain
21,21,Victoria,HK,22.2855,114.1577,86.38,84,16,1.99,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                174
City                   174
Country                174
Lat                    174
Lng                    174
Max Temp               174
Humidity               174
Cloudiness             174
Wind Speed             174
Current Description    174
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City ID                174
City                   174
Country                174
Lat                    174
Lng                    174
Max Temp               174
Humidity               174
Cloudiness             174
Wind Speed             174
Current Description    174
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hithadhoo,MV,80.60,light rain,-0.6000,73.0833,
4,Hilo,US,82.80,light rain,19.7297,-155.0900,
6,Trairi,BR,80.28,few clouds,-3.2778,-39.2689,
7,Broome,US,85.69,scattered clouds,42.2506,-75.8330,
8,Butaritari,KI,82.08,moderate rain,3.0707,172.7902,
12,Faanui,PF,78.01,clear sky,-16.4833,-151.7500,
13,Vaini,TO,75.47,broken clouds,-21.2000,-175.2000,
14,Albany,US,85.03,broken clouds,42.6001,-73.9662,
15,Kapaa,US,82.72,light rain,22.0752,-159.3190,
21,Victoria,HK,86.38,few clouds,22.2855,114.1577,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat}, {lng}"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")

hotel_df.head(10)        

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hithadhoo,MV,80.60,light rain,-0.6000,73.0833,Scoop Guest House
4,Hilo,US,82.80,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Trairi,BR,80.28,few clouds,-3.2778,-39.2689,janelas do mar
7,Broome,US,85.69,scattered clouds,42.2506,-75.8330,Chenango Valley State Park
8,Butaritari,KI,82.08,moderate rain,3.0707,172.7902,Isles Sunset Lodge
12,Faanui,PF,78.01,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
13,Vaini,TO,75.47,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
14,Albany,US,85.03,broken clouds,42.6001,-73.9662,
15,Kapaa,US,82.72,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
21,Victoria,HK,86.38,few clouds,22.2855,114.1577,Mini Hotel Central


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace("", np.nan)
clean_hotel_df.dropna(inplace=True)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Hithadhoo,MV,80.60,light rain,-0.6000,73.0833,Scoop Guest House
4,Hilo,US,82.80,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Trairi,BR,80.28,few clouds,-3.2778,-39.2689,janelas do mar
7,Broome,US,85.69,scattered clouds,42.2506,-75.8330,Chenango Valley State Park
8,Butaritari,KI,82.08,moderate rain,3.0707,172.7902,Isles Sunset Lodge
12,Faanui,PF,78.01,clear sky,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
13,Vaini,TO,75.47,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
15,Kapaa,US,82.72,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
21,Victoria,HK,86.38,few clouds,22.2855,114.1577,Mini Hotel Central
22,Luwuk,ID,75.29,broken clouds,-0.9516,122.7875,Hotel Santika Luwuk


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} &#176F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(25.0,5.0), zoom_level=1.4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))